In [7]:
from scipy.io import loadmat
import numpy as np

In [8]:
# time of precomputation. average iteration, AUC and t for best pair after cv for each epsilon 
epsilon_list = [0.01, 0.05, 0.1, 0.5, 1.0, 10.0, -1]
group_list = np.arange(10)
before_precoputation_time_dic = {}
before_iteration_time_dic = {}
after_t_dic = {}
after_AUC_dic = {}

for group in group_list:
    before = loadmat(f'Group{group}.mat')['Res']
    before_precoputation_time_dic[group] = before[:,0]
    before_iteration_time_dic[group] = before[:,1] / 5
        
    for idx, eps in enumerate(epsilon_list):
        after = loadmat(f'Prox_ROC_g{group}_epsilon{idx+1}.mat')['Prox_ROC']
        if group not in after_t_dic:
            after_t_dic[group] = [after[0][0][6][0][0]]
            after_AUC_dic[group] = [after[0][0][3][0][0]]
        else:
            after_t_dic[group].append(after[0][0][6][0][0])
            after_AUC_dic[group].append(after[0][0][3][0][0])


In [9]:
before_iteration_time_dic

{0: array([0.97492526, 0.97731593, 0.9772719 , 0.97441674, 0.97574821,
        0.97706064, 0.99177131]),
 1: array([0.95907622, 0.95686161, 0.95742712, 0.9571817 , 0.95760087,
        0.96020358, 0.97628334]),
 2: array([0.95613688, 0.95401781, 0.95059008, 0.95266423, 0.95195844,
        0.95670073, 0.97808226]),
 3: array([0.94926979, 0.94918854, 0.94974502, 0.94862095, 0.94574681,
        0.94625316, 0.96448607]),
 4: array([0.95871323, 0.95624495, 0.95028586, 0.94966624, 0.95382492,
        0.95265649, 0.97367629]),
 5: array([0.97732161, 0.95876587, 0.95542396, 0.95565405, 0.95745492,
        0.95700955, 0.97983935]),
 6: array([0.95539541, 0.95605737, 0.96004995, 0.96163963, 0.96091536,
        0.95663496, 0.97773761]),
 7: array([0.95104831, 0.9482382 , 0.95132698, 0.94976356, 0.94781378,
        0.95066212, 0.96128941]),
 8: array([0.97991329, 0.97755607, 0.97400233, 0.9710791 , 0.97219181,
        0.97480312, 0.99423791]),
 9: array([1.01920069, 1.02618521, 1.01809987, 1.016514

In [10]:
loadmat(f'Prox_ROC_g0_epsilon1.mat')['Prox_ROC'][0][0][6][0][0]

0.965208

In [11]:
# value of the largest eigenvalue for X^TX and the computation time

data = loadmat('UCF101_group_epsilon_eig.mat')['group_epsilon_eig']
data_dic = {
    'value': [x[0][0] for x in data[0]],
    'time': [x[0][1] for x in data[0]]
}

In [12]:
import pandas as pd
group_list = np.arange(10)
wrtier = pd.ExcelWriter('UCF_epsilon_exp.xlsx')
pd.DataFrame(before_iteration_time_dic, index=epsilon_list).to_excel(wrtier, sheet_name='Iteration time(per fold)')
pd.DataFrame(before_precoputation_time_dic, index=epsilon_list).to_excel(wrtier, sheet_name='Precomputation time(in CV)', index=epsilon_list)
pd.DataFrame(after_AUC_dic, index=epsilon_list).to_excel(wrtier, sheet_name='AUC(after CV)', index=epsilon_list)
pd.DataFrame(after_t_dic, index=epsilon_list).to_excel(wrtier, sheet_name='t(after CV)', index=epsilon_list)
pd.DataFrame(data_dic, index=group_list).to_excel(wrtier, sheet_name='epsilon_computation')
wrtier.save()